# Retreival based Chatbot

In [1]:
import numpy as np
import nltk
import random
import string

import bs4 as bs
import urllib.request
import re

In [2]:
raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Cuisine')
raw_html = raw_html.read()

In [3]:
article_html = bs.BeautifulSoup(raw_html, 'lxml')

article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [4]:
# Step 2 : Text Preprocessing and Helper Function

article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
article_text = re.sub(r'\s+', ' ', article_text)

In [5]:
article_sentences = nltk.sent_tokenize(article_text)
article_words = nltk.word_tokenize(article_text)

In [6]:
wnlemmatizer = nltk.stem.WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [wnlemmatizer.lemmatize(token) for token in tokens]

punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

In [7]:
greeting_inputs = ("hey", "good morning", "good evening", "morning", "evening", "hi", "whatsup", "hello")
greeting_responses = ["hey", "hey hows you?", "*nods*", "hello, how you doing", "hello", "Welcome, I am good and you"]

def generate_greeting_response(greeting):
    for token in greeting.split():
        if token.lower() in greeting_inputs:
            return random.choice(greeting_responses)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
def generate_response(user_input):
    tennisrobo_response = ''
    article_sentences.append(user_input)

    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')
    all_word_vectors = word_vectorizer.fit_transform(article_sentences)
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
    similar_sentence_number = similar_vector_values.argsort()[0][-2]

    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        tennisrobo_response = tennisrobo_response + "I am sorry, I could not understand you"
        return tennisrobo_response
    else:
        tennisrobo_response = tennisrobo_response + article_sentences[similar_sentence_number]
        return tennisrobo_response

In [10]:
word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')
all_word_vectors = word_vectorizer.fit_transform(article_sentences)

C:\Users\singhegm\Anaconda_new\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [11]:
similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

In [12]:
similar_sentence_number = similar_vector_values.argsort()[0][-2]

In [13]:
continue_dialogue = True
print("Hello, I am your friend Foodie. You can ask me any question regarding Cuisines:")
while(continue_dialogue == True):
    human_text = input()
    human_text = human_text.lower()
    if human_text != 'bye':
        if human_text == 'thanks' or human_text == 'thank you very much' or human_text == 'thank you':
            continue_dialogue = False
            print("Foodie : Most welcome")
        else:
            if generate_greeting_response(human_text) != None:
                print("Foodie : " + generate_greeting_response(human_text))
            else:
                print("Foodie : ", end="")
                print(generate_response(human_text))
                article_sentences.remove(human_text)
    else:
        continue_dialogue = False
        print("Foodie : Good bye and take care of yourself...")

Hello, I am your friend Foodie. You can ask me any question regarding Cuisines:
Hello
Foodie : hey
what are cuisines?
Foodie : the regional cuisines are north american cuisine, mexican cuisine, central american cuisine, south american cuisine, and caribbean cuisine.
cuisines
Foodie : the regional cuisines are north american cuisine, mexican cuisine, central american cuisine, south american cuisine, and caribbean cuisine.
best cuisine
Foodie : the regional cuisines are north american cuisine, mexican cuisine, central american cuisine, south american cuisine, and caribbean cuisine.
Tanzania
Foodie : typical ethiopian and eritrean cuisine: injera (thin pancake-like bread) and several kinds of wat (stew) a ramadan dinner in tanzania yassa is a popular dish throughout west africa prepared with chicken or fish.
Guangdong's
Foodie : due to guangdong's location on the southern coast of china, fresh live seafood is a specialty in cantonese cuisine.
Chiang Mai
Foodie : a market stall at thanin m